## Welcome to the FROST Notebook
We'll be taking a look at FROST internals through implementing FROST.
FROST (Flexible Round-Optimized Schnorr Threshold) is a cryptographic protocol that allows multiple parties to collaboratively sign a message while maintaining security against various types of attacks. It utilizes threshold cryptography, where a signature can be generated only when a threshold number of parties cooperate.
We'll divide the discussion of FROST into three steps: key generation, signing, and verification.


In [57]:
## Lets set up our global constants 
# Prime order of curve
P = 2**256 - 2**32 - 977
# The order of the base point (also known as the generator point G),
# which is the number of distinct points on the curve that can be generated by
# repeatedly adding the base point to itself.
Q = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
# Generator point Cordinates
G_x = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
G_y = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
# We'll use these cordinates to set up ECC point in a later step
# string context - protect against replay attacks
CONTEXT = b'FROST-BIP340'


In [58]:
# Define ECC point class
class Point:
        """Class representing an elliptic curve point."""
        def __init__(self, x=float('inf'), y=float('inf')):
            self.x = x
            self.y = y

        @classmethod
        def secret_deserialize(self, hex_public_key):
            hex_bytes = bytes.fromhex(hex_public_key)
            is_even = hex_bytes[0] == 2
            x_bytes = hex_bytes[1:]
            x = int.from_bytes(x_bytes, 'big')
            y_squared = (pow(x, 3, P) + 7) % P
            y = pow(y_squared, (P + 1) // 4, P)

            if y % 2 == 0:
                even_y = y
                odd_y = (P - y) % P
            else:
                even_y = (P - y) % P
                odd_y = y
            y = even_y if is_even else odd_y

            return self(x, y)

        def secret_serialize(self):
            # Return compressed key
            prefix = b'\x02' if self.y % 2 == 0 else b'\x03'

            return prefix + self.x.to_bytes(32, 'big')

        @classmethod
        def xonly_deserialize(self, hex_public_key):
            hex_bytes = bytes.fromhex(hex_public_key)
            x = int.from_bytes(hex_bytes, 'big')
            y_squared = (pow(x, 3, P) + 7) % P
            y = pow(y_squared, (P + 1) // 4, P)

            if y % 2 != 0:
                y = (P - y) % P

            return self(x, y)

        def xonly_serialize(self):
            return self.x.to_bytes(32, 'big')

        # point at infinity
        def is_zero(self):
            return self.x == float('inf') or self.y == float('inf')

        def __eq__(self, other):
            return self.x == other.x and self.y == other.y

        def __ne__(self, other):
            return not self == other

        def __neg__(self):
            if self.is_zero():
                return self

            return self.__class__(self.x, P - self.y)

        # Double point
        def dbl(self):
            x = self.x
            y = self.y
            s = (3 * x * x * pow(2 * y, P - 2, P)) % P
            sum_x = (s * s - 2 * x) % P
            sum_y = (s * (x - sum_x) - y) % P

            return self.__class__(sum_x, sum_y)

        def __add__(self, other):
            if self == other:
                return self.dbl()
            if self.is_zero():
                return other
            if other.is_zero():
                return self
            if self.x == other.x and self.y != other.y:
                return self.__class__()
            s = ((other.y - self.y) * pow(other.x - self.x, P - 2, P)) % P
            sum_x = (s * s - self.x - other.x) % P
            sum_y = (s * (self.x - sum_x) - self.y) % P

            return self.__class__(sum_x, sum_y)

        def __rmul__(self, scalar):
            p = self
            r = self.__class__()
            i = 1

            while i <= scalar:
                if i & scalar:
                    r = r + p
                p = p.dbl()
                i <<= 1

            return r

        def __str__(self):
            if self.is_zero():
                return '0'
            return 'X: 0x{:x}\nY: 0x{:x}'.format(self.x, self.y)

        def __repr__(self) -> str:
            return self.__str__()

In [59]:
# Create ECC Generator point
G = Point(G_x, G_y)


## Distributed Key Generation (DKG)
Pedersen-style Distributed Key Generation (DKG) is a cryptographic protocol used to securely generate and distribute cryptographic keys in a decentralized manner. The protocol is named after its creator, Torben Pedersen. DKG is particularly useful in scenarios where a group of parties need to collaboratively generate a shared secret key while ensuring that no subset of participants can determine the key on their own.

The key goal of DKG is to ensure that the final generated key is both unpredictable and secure, even in the presence of malicious participants or communication failures. This is achieved through a series of interactive steps that involve multiple participants working together to contribute information and perform computations.

Here's a simplified overview of how a Pedersen-style DKG protocol might work:

- Setup: Before the protocol begins, participants agree on certain parameters and cryptographic assumptions. These might include the choice of a mathematical group (such as an elliptic curve group), security parameters, and other relevant settings.
- Commitment Phase: Each participant generates a commitment to their secret share (a partial key) without revealing the actual value. These commitments are shared with the entire group. This phase ensures that participants are committed to their shares and prevents them from altering their values later in the protocol.
- Communication and Verification Phases: In a series of rounds, participants share information and perform verifiable computations. They exchange commitments, proofs, and other cryptographic constructs to ensure that their commitments are consistent with the protocol rules and that everyone is following the correct steps. These phases might involve various mathematical operations and interactive proofs.
- Reconstruction Phase: After all the necessary computations and interactions, the protocol reaches a point where the final shared key can be reconstructed. This involves combining the secret shares contributed by each participant to compute the group's shared secret key. Importantly, no individual participant can determine the key on their own.


For the purpose of the lesson plan we'll be setting up a 2 out of 3 (2:3) threshold FROST setup.

In [60]:
THRESHOLD = 2
N = 3

In [61]:
import secrets
from hashlib import sha256

def new_polynomial(threshold):
    # Generate Shamir polynomial with random coefficients, and with degree
    # equal to the threshold minus one.
    coefficients = [secrets.randbits(256) % Q for _ in range(threshold)]
    return coefficients

Initiating the Distributed Key Generation (DKG) procedure involves every participant generating a set of random numbers. A set with length = N-1. These random numbers are then utilized to construct a mathematical entity known as a "Shamir polynomial".
In the context of a 2-out-of-3 (2:3), Alice's polynomial would be $f_i(x) = a_0x^0 + a_1x^1 = a_0 + a_1x^1$. Where $i$ is Alice's index in the multiset. Therefore the numbers Alice would generate would be the coeffecients of her polynomial. i.e $a_0$ and $a_1$. 

In [62]:
p1 = new_polynomial(THRESHOLD)
p2 = new_polynomial(THRESHOLD)
p3 = new_polynomial(THRESHOLD)

p1_commitments = [coefficient * G for coefficient in p1]
p2_commitments = [coefficient * G for coefficient in p2]
p3_commitments = [coefficient * G for coefficient in p3]

We take the polynomial’s coefficients, and raise them to $g$ to produce the public commitments. The only value relevant for producing a public key is the commitment derived from the first coefficient. We'll get more into that later.
To recap so far we have 3 participants and 3 polynomials.
* Alice -> $f_1(x) = a_0x^0 + a_1x^1$
* Bob -> $f_2(x) = a_0x^0 + a_1x^1$
* Carol -> $f_3(x) = a_0x^0 + a_1x^1$


Each participant have public commitments which are just the coeffecients ($a_i$) multiplied by the generator. $g^{a_i}$





Next up is the commitment phase of DKG.
> Each participant generates a commitment to their secret share (a partial key) without revealing the actual value. These commitments are shared with the entire group. This phase ensures that participants are committed to their shares and prevents them from altering their values later in the protocol 

In [63]:
# Participant needs to commit to their partial key
# And commit to their index in the signing session
# This is done by signing a nonce and their index
def proof_of_knowledge(index, coefficients):
    nonce = secrets.randbits(256) % Q
    nonce_point = nonce * G
    # a_0 (first coef) is always the partial private key
    secret = coefficients[0]
    secret_commitment = secret * G

    # The challenge below is composed of the
    # Index in multiset, starting at 1
    # The frost tag
    # Your partial secret key (a_0)
    # Your nonce
    challenge_input = index.to_bytes(1, 'big') + CONTEXT + secret_commitment.secret_serialize() + nonce_point.secret_serialize()
    challenge_hash = sha256(challenge_input)
    challenge_hash_bytes = challenge_hash.digest()
    challenge_hash_int = int.from_bytes(challenge_hash_bytes, 'big')
    
    s = (nonce + secret * challenge_hash_int) % Q

    return [nonce_point, s]


To provide a commitment to FROST signing session we produce a signature with our partial key.
The challenge is composed of $c = H(i | Ω | a_0 | K)$
Where $K$ is public nonce point and $Ω$ is the FROST tag. Note that the proof primarily involves verifying familiarity with the initial coefficient ($a_0$) without disclosing its actual value.

In [64]:
p1_proof = proof_of_knowledge(1, p1)
p2_proof = proof_of_knowledge(2, p2)
p3_proof = proof_of_knowledge(3, p3)

In [65]:
# We want to verify a participant verifies knowledge of their coeffcients
# Proof is a normal schnorr signature (s,r)
def verify_proof_of_knowledge(index, proof, first_coeffecient):
    [nonce_point, s] = proof
    # Recalculate the challenge from above
    challenge_input = index.to_bytes(1, 'big') + CONTEXT + first_coeffecient.secret_serialize() + nonce_point.secret_serialize()
    challenge_hash_bytes = sha256(challenge_input).digest()
    challenge_hash_int = int.from_bytes(challenge_hash_bytes, 'big')

    return nonce_point == (s * G) + (Q - challenge_hash_int) * first_coeffecient
    

We verify each proof by checking if $R = s * G + C * g^{a_0}$. Note this is not any different than the naive Shnorr verification algorithm 

In [66]:
# Verify proofs
# Each participant needs to do this for each proof provided
assert(verify_proof_of_knowledge(1, p1_proof, p1_commitments[0]) == True)
assert(verify_proof_of_knowledge(2, p2_proof, p2_commitments[0]) == True)
assert(verify_proof_of_knowledge(3, p3_proof, p3_commitments[0]) == True)

assert(not verify_proof_of_knowledge(2, p1_proof, p1_commitments[0]))
assert(not verify_proof_of_knowledge(1, p1_proof, p2_commitments[0]))


rior to the reconstruction of the shared aggregated secret or the aggregated public key, every participant must possess their respective partial share. A partial share is the result of summing up the evaluations of each participant's polynomial at every index within the multiset. Essentially, a share represents a fractional element of the aggregated secret. A single share alone is insufficient to derive the complete secret, but a specified set or the threshold number of shares has the capability to do so.
For example, $s_i = \sum_{j=1}^{n} f_j(i)$ Where $i$ is the particiapants index.

In essence:
* Alice sums up $f_1(1) + f_2(1) + f_3(1) $ to create $s_1$ 
* Bob sums up $f_1(2) + f_2(2) + f_3(2) $ to create $s_2$
* Carol sums up $f_1(3) + f_2(3) + f_3(3) $ to create $s_3$


![Shares]("images/shamir_sharing.png")

In [67]:
# Lets generate aggregate shares
# Coefs here are ints, not ecc points
# However the partial share is a point on the ecc graph
def evaluate_polynomial(index, participant_coefficients):
    share = 0
    for i in range(len(participant_coefficients) - 1, -1, -1):
        share = share * index + participant_coefficients[i]

    return share % Q


def generate_share(n, participant_coefficients):
    # Evaluate a participants poly at each index
    # Remeber we index at 1
    shares = [evaluate_polynomial(i, participant_coefficients) for i in range(1, n + 1)]
    return shares


In [68]:
p1_share = generate_share(N, p1)
p2_share = generate_share(N, p2)
p3_share = generate_share(N, p3)

In [69]:
# lets verify each share was calculated correctly
def verify_share(shares, coefficient_commitments, index):
    expected_y_commitment = Point()
    for coef_index, coef in enumerate(coefficient_commitments):
        expected_y_commitment = expected_y_commitment + \
                    ((index ** coef_index % Q) * coef)
    return shares * G == expected_y_commitment

In [70]:
# Is our aggregated share for participant #1 correct
verify_share(p1_share[0], p1_commitments, 1)
# TODO verify shares for other participants

True

From here calculating the aggregated share is simply the sumation of the partial shares. Each participant will calculate it differently, by putting their partial share first and store is seperately.

In [71]:
def aggregate_shares(participant_share, shares):
    aggregate_share = participant_share
    for share in shares:
        aggregate_share = aggregate_share + share
    return aggregate_share % Q

In [72]:

p1_agg = aggregate_shares(p1_share[0],[p2_share[0], p3_share[0]])
p2_agg = aggregate_shares(p2_share[1],[p1_share[1], p3_share[1]])
p3_agg = aggregate_shares(p3_share[2],[p2_share[2], p1_share[2]])


As previously indicated, when it comes to generating the public key, our focus is solely on the initial coefficient. In other words, $a_0$, which repersents the exact point of intersection between the polynomial and the y-axis. 

The aggregate public key is therfore the summation of all y intercept points.
$P = \sum_{i=1}^{n} g^{f(0)}$

In [73]:
# x_0 for each participant polynomials
def derive_public_key(first_coeffecients):
    public_key = first_coeffecients[0]
    for commitment in first_coeffecients[1:]:
        public_key = public_key + commitment

    return public_key



Note: Any combination of the participant coeffecient should produce the same outcome. We'll leave this as an exercise for the reader

In [74]:
aggregate_public_key = derive_public_key([p1_commitments[0], p2_commitments[0], p3_commitments[0]])
aggregate_public_key


X: 0x130e792ddcd0991f4073726191b9a2059965ba8e5d4ac96234f96af13bdb1a1f
Y: 0xfe10d1479b898350b9264d5b191a4fa92a0cd136a97115428c623b3ffea8fbb8

Job done! We have a aggregate public key and shares used to provide a threshold signature. Let's throw a party...


Not just yet. Before we sign and verify lets ensure we can reconstruct the aggregate public key given a set of aggregate shares formed by 2:3 participants. 

To do this we need to compute a coefficient that when multiplied by the corresponding polynomial value at a specific index, contributes to the reconstruction of the original polynomial based on the given participant indexes and the index of interest.

$L = \frac{{\prod_{i=1, i \neq my\_index}^{n} index_i}}{{\prod_{i=1, i \neq my\_index}^{n} (index_i - my\_index)}} \cdot (index\_denominator^{Q - 2} \mod Q)$

Where:
- $( index\_denominator = \prod_{i=1, i \neq my\_index}^{n} (index_i - my\_index)$


In summary, the function calculates the Lagrange coefficient $L$ by dividing the product of all participant indices (excluding my_index) by the product of the differences between each participant index and my_index. It then multiplies this by the modular inverse of index_denominator modulo $Q$, resulting in the final Lagrange coefficient.


In [75]:
 def lagrange_coefficient(participant_indexes, my_index):
    numerator = 1
    denominator = 1
    for index in participant_indexes:
        if index == my_index:
            continue
        numerator = numerator * index
        denominator = denominator * (index - my_index)
    return (numerator * pow(denominator, Q - 2, Q)) % Q

In [76]:
# Lets reconstruct the secret
# First we construct a lagrange coefficient for the first participant
l1 = lagrange_coefficient([2], 1)
# And one for the second participant
l2 = lagrange_coefficient([1], 2)
# Verify we get the same aggregate public key
secret = ((p1_agg * l1) + (p2_agg * l2)) % Q
assert(secret * G == aggregate_public_key)

# Lets repeat the same exercise for the third and second participants
l3 = lagrange_coefficient([2], 3)
l2 = lagrange_coefficient([3], 2)
secret = ((p3_agg * l3) + (p2_agg * l2)) % Q
assert(secret * G == aggregate_public_key)

Note here that we are able to reconstruct aggregated public key by simply $g^s$. In the previous steps we needed to sum all first coeffecients. So it worked!

To recap:
The overall process of DKG can be outlined as follows:
* Each participant initiates the creation of a Shamir polynomial, establishing the polynomial's coefficients.
* They subsequently produce their respective shares and corresponding commitments.
* The shares and commitments are then disseminated, allowing each individual participant to conduct verification.
* By combining the commitments, the public key is computed, serving as a culmination of this procedure.

## Signing

For the signing process, we will follow a two-step protocol. In this protocol, all participants undertake the following actions:
1. Generate pairs of nonces.
2. Share these nonce pairs with each other or with a central coordinator.
3. Subsequently, produce partial signatures for a universally shared message.

In [77]:
# Lets first define utility method for generating random nonce pairs
def generate_nonces():
    nonce_pair = [secrets.randbits(256) % Q, secrets.randbits(256) % Q]
    return nonce_pair


In [78]:
p1_nonce_pair = generate_nonces()
p2_nonce_pair = generate_nonces()
p3_nonce_pair = generate_nonces()

p1_nonce_point = [p1_nonce_pair[0] * G, p1_nonce_pair[1] * G]
p2_nonce_point = [p2_nonce_pair[0] * G, p2_nonce_pair[1] * G]
p3_nonce_point = [p3_nonce_pair[0] * G, p3_nonce_pair[1] * G]


Subsequently, a signature aggregator is designated, which might be a participant or a third-party coordinator.

The role of this signature aggregator entails collecting the previously generated nonce commitments from each signer $i$, resulting in a set $B = \{(i, D_i, E_i)\}$, where $D$ and $E$ signify the nonce pairs for each participant.

With this accomplished, the signature aggregator becomes capable of disseminating both the message $m$ and the set $B$ to every participant engaged in the signing process.

Each signing participant then embarks on the task of verifying the message and calculating a binding value $p_i = H(i | m | B)$.

In [79]:
def binding_value(index, message, nonce_commitment_pairs, participant_indexes):
    bv = sha256()
    nonce_commitment_pairs_bytes = []
    for index in participant_indexes:
        participant_pair = nonce_commitment_pairs[index-1]
        participant_pair_bytes = b''.join([commitment.secret_serialize() for commitment in participant_pair])
        nonce_commitment_pairs_bytes.append(participant_pair_bytes)
    nonce_commitment_pairs_bytes = b''.join(nonce_commitment_pairs_bytes)
    pre_image = index.to_bytes(1, 'big') + message + nonce_commitment_pairs_bytes
    bv = sha256(pre_image)
    binding_value_bytes = bv.digest()

    return int.from_bytes(binding_value_bytes, 'big')

In essence the binding value is a commitment to the message, signing set, and your index in the signing set.
Binding values are used to derive the group commitment $R$.

Defined as $R = \prod_{i=1, D_i * (E_i)^{p*i}}^{n}$

In [80]:
def group_commitment(message, nonce_commitment_pairs, participant_indexes):
    gc = Point()
    for index in participant_indexes:
        bv = binding_value(index, message, nonce_commitment_pairs, participant_indexes)
        first_commitment = nonce_commitment_pairs[index-1][0]
        second_commitment = nonce_commitment_pairs[index-1][1]
        gc = gc + (first_commitment + (bv * second_commitment))
    return gc

And finally our challenge $c$, defined as

$c = H(Ω|Ω|R|Y|m)$, where $Y$ is the aggregate public key and $Ω$ is the FROST tag

In [81]:
def challenge_hash(nonce_commitment, aggregate_public_key, message):
    tag_hash = sha256(b'BIP0340/challenge').digest()
    challenge_hash = sha256()
    challenge_hash.update(tag_hash)
    challenge_hash.update(tag_hash)
    challenge_hash.update(nonce_commitment.xonly_serialize())
    challenge_hash.update(aggregate_public_key.xonly_serialize())
    challenge_hash.update(message)
    challenge_hash_bytes = challenge_hash.digest()

    return int.from_bytes(challenge_hash_bytes, 'big') % Q

Partial signature are defined as
$z_i = d_i + (e_i + p_i) + \lambda_i * s_i * c$.


In [82]:
def sign(message, nonce_commitment_pairs, participant_indexes, signing_nonce_pair, signer_index, aggregate_public_key, aggregate_share):
    gc = group_commitment(message, nonce_commitment_pairs, participant_indexes)
    challenge = challenge_hash(gc, aggregate_public_key, message)
    [first_nonce, second_nonce] = signing_nonce_pair

    bv = binding_value(signer_index, message, nonce_commitment_pairs, participant_indexes)
    
    lagrange = lagrange_coefficient(participant_indexes, my_index=signer_index)

    # Negate nonce pair if group commitment is odd
    if gc.y % 2 != 0:
        first_nonce = Q - first_nonce
        second_nonce = Q - second_nonce
    
    # Negate aggregate share if aggregate public key point is odd
    if aggregate_public_key.y % 2 != 0:
        aggregate_share = Q - aggregate_share

    return (first_nonce + (second_nonce * bv) + lagrange * aggregate_share * challenge) % Q

In [83]:
# Lets sign with the first and second participants
msg = b'craigwrightisnotsatoshi!'
participant_indexes = [1, 2]
nonce_commitment_pairs = [p1_nonce_point, p2_nonce_point]

s1 = sign(
    message=msg,
    nonce_commitment_pairs=nonce_commitment_pairs,
    participant_indexes=participant_indexes,
    signing_nonce_pair=p1_nonce_pair,
    signer_index=1,
    aggregate_public_key=aggregate_public_key,
    aggregate_share=p1_agg
)

s2 = sign(
    message=msg,
    nonce_commitment_pairs=nonce_commitment_pairs,
    participant_indexes=participant_indexes,
    signing_nonce_pair=p2_nonce_pair,
    signer_index=2,
    aggregate_public_key=aggregate_public_key,
    aggregate_share=p2_agg
)


Once a signature aggregator has a threshold of partial signatures, they can produce the final aggregate signature by summing all partial signature.

In [87]:

def aggregate_signatures(signature_shares, gc, challenge):
    # TODO: verify each signature share
    nonce_commitment = gc.xonly_serialize()
    z = (sum(signature_shares) % Q).to_bytes(32, 'big')

    return bytes.fromhex((nonce_commitment + z).hex())

In [88]:
gc = group_commitment(msg, nonce_commitment_pairs, participant_indexes)
challenge = challenge_hash(gc, aggregate_public_key, msg)
agg_sig = aggregate_signatures([s1, s2], gc=gc, challenge=challenge)
nonce_commitment = Point.xonly_deserialize(agg_sig[0:32].hex())
z = int.from_bytes(agg_sig[32:64], 'big')

# verify
# Negate Y if Y.y is odd
if aggregate_public_key.y % 2 != 0:
    aggregate_public_key = -aggregate_public_key

assert(nonce_commitment == (z * G) +(Q - challenge) * aggregate_public_key)